In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
local_path = '/Users/fabienaugsburger/myCloud/Unil/Master/Thesis/Datasets/Curnagl/ERA5_1990_2m_dewpoint_temperature.nc'

dew_point_xr = xr.open_dataset(local_path)
dew_point_xr

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 2920)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-12-31T21:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-10-13 21:02:30 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [29]:
# import 1st track

#head = ['lon_east', 'lon_west', 'lat_south', 'lat_north']
#track_1 = pd.read_csv('tc_irad_tracks/tc_irad_1.txt')
#track_1.columns = head

# import all tracks

for i in range(0,8):
    locals()['track_' + str(i+1)] = pd.read_csv('tc_irad_tracks/tc_irad_' + str(i+1) + '.txt')
    

In [30]:
# slice the time dimension

dates = pd.read_csv('storms_start_end.csv', sep=',')

'''
start_first = dates['start_date'][0]
end_first = dates['end_date'][0]
'''
for i in range(0,8):
    start_temp = dates['start_date'][i]
    end_temp = dates['end_date'][i]

    # Create a new dataset for each iteration
    new_dataset = xr.Dataset({
        'd2m': dew_point_xr['d2m'],
        # Add other variables as needed
    })

    # Optionally, you can update the time dimension for the new dataset
    new_dataset = new_dataset.sel(time=slice(start_temp, end_temp))

    # Dynamically name the dataset variable
    locals()[f"dew_point_xr_{i}"] = new_dataset

'''
dew_point_xr = dew_point_xr.sel(time=slice(start_first, end_first))
dew_point_xr
'''

'\ndew_point_xr = dew_point_xr.sel(time=slice(start_first, end_first))\ndew_point_xr\n'

In [31]:
dew_point_xr_0

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 62)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1990-01-21T03:00:00 ... 1990-01-28T18:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 ...

In [6]:
# create new pd dataframe
'''
first_slice = track_1.iloc[0]

test_dp     = dew_point_xr.sel(longitude=slice(first_slice['lon_east'], first_slice['lon_west']), latitude=slice(first_slice['lat_north'], first_slice['lat_south']))
test_dp'''


"\nfirst_slice = track_1.iloc[0]\n\ntest_dp     = dew_point_xr.sel(longitude=slice(first_slice['lon_east'], first_slice['lon_west']), latitude=slice(first_slice['lat_north'], first_slice['lat_south']))\ntest_dp"

In [7]:
# plot the first slice with matplotlib
'''
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 5))
plt.pcolormesh(test_dp.longitude, test_dp.latitude, test_dp.d2m[0,:,:], cmap='coolwarm')'''

"\nimport matplotlib.pyplot as plt\n\n\nplt.figure(figsize=(10, 5))\nplt.pcolormesh(test_dp.longitude, test_dp.latitude, test_dp.d2m[0,:,:], cmap='coolwarm')"

In [8]:
# check if it's correct
'''
lon_test = np.asanyarray(dew_point_xr.longitude[:])
lat_test = np.asanyarray(dew_point_xr.latitude[:])


closest_lon_w =np.abs(lon_test - first_slice['lon_west']).argmin()
closest_lon_e = np.abs(lon_test - first_slice['lon_east']).argmin()
closest_lat_s = np.abs(lat_test - first_slice['lat_south']).argmin()
closest_lat_n = np.abs(lat_test - first_slice['lat_north']).argmin()

lon_test[closest_lon_w]
lat_test[closest_lat_s]
'''
# works

"\nlon_test = np.asanyarray(dew_point_xr.longitude[:])\nlat_test = np.asanyarray(dew_point_xr.latitude[:])\n\n\nclosest_lon_w =np.abs(lon_test - first_slice['lon_west']).argmin()\nclosest_lon_e = np.abs(lon_test - first_slice['lon_east']).argmin()\nclosest_lat_s = np.abs(lat_test - first_slice['lat_south']).argmin()\nclosest_lat_n = np.abs(lat_test - first_slice['lat_north']).argmin()\n\nlon_test[closest_lon_w]\nlat_test[closest_lat_s]\n"

In [9]:
#slice with new boundaries
'''
plt.pcolormesh(np.flipud(dew_point_xr['d2m'][0,176:208,1157:1189]), cmap='coolwarm')
plt.colorbar()'''

"\nplt.pcolormesh(np.flipud(dew_point_xr['d2m'][0,176:208,1157:1189]), cmap='coolwarm')\nplt.colorbar()"

In [10]:
#lat_test[208]

In [33]:
# iterate through each time step

var_out = []
'''
for i in range(0, len(track_1)):
    lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_1.iloc[i]

    lon_test = np.asanyarray(dew_point_xr.longitude[:])
    lat_test = np.asanyarray(dew_point_xr.latitude[:])

    closest_lon_w =np.abs(lon_test - lon_w_temp).argmin()
    closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
    closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
    closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

    temp_ds = np.asarray(dew_point_xr['d2m'])

    var_out.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])
'''
var_out
for j in range (0,7):
    track_temp = pd.read_csv(f'tc_irad_tracks/tc_irad_{j+1}.txt')
    dew_point_temp = locals()[f"dew_point_xr_{j}"]
    var_out_temp = []
    for i in range(0, len(track_temp)):
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_temp.iloc[i]

        lon_test = np.asanyarray(dew_point_temp.longitude[:])
        lat_test = np.asanyarray(dew_point_temp.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        closest_lon_w_coor = lon_test[closest_lon_w]
        closest_lon_e_coor = lon_test[closest_lon_e]
        closest_lat_s_coor = lat_test[closest_lat_s]
        closest_lat_n_coor = lat_test[closest_lat_n]

        # Use .roll to handle the 0°/360° boundary

        if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
            roll_shift = {'longitude':int(round(closest_lon_w_coor,0)), 'longitude': int(round(closest_lon_e_coor,0))}
            dew_point_temp_rolled = dew_point_temp.roll(roll_shift, roll_coords=True)
        else:
            dew_point_temp_rolled = dew_point_temp
        # Slice the dataset based on the rolled longitudes and latitudes
        temp_ds_time = dew_point_temp_rolled['d2m'].isel(time=i)
        temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                           longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

        #temp_ds = np.asarray(dew_point_temp['d2m'])

        #var_out_temp.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])
        var_out_temp.append(temp_ds)

    locals()[f"var_out_{j+1}"] = var_out_temp


In [14]:
# test

'''for j in range(0, 4):
    track_temp = pd.read_csv(f'tc_irad_tracks/tc_irad_{j+1}.txt')
    dew_point_temp = locals()[f"dew_point_xr_{j}"]
    var_out_temp = []

    for i in range(0, len(track_temp)):
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_temp.iloc[i]

        lon_test = np.asanyarray(dew_point_temp.longitude[:])
        lat_test = np.asanyarray(dew_point_temp.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        # Check if the indices create a valid slice
        if closest_lat_s > closest_lat_n and closest_lon_w < closest_lon_e:
            # Use .roll to handle the 0°/360° boundary
            roll_shift = {'longitude': closest_lon_e, 'longitude': closest_lon_w}
            dew_point_temp_rolled = dew_point_temp.roll(roll_shift, roll_coords=True)

            # Slice the dataset based on the rolled longitudes and latitudes
            temp_ds = dew_point_temp_rolled['d2m'].isel(time=i, latitude=slice(closest_lat_n, closest_lat_s),
                                                        longitude=slice(closest_lon_e, closest_lon_w)).values

            var_out_temp.append(temp_ds)
        else:
            print(f"Invalid indices for square at time index {i} in track {j+1}")

    locals()[f"var_out_{j+1}"] = var_out_temp
'''

roll_test = {'longitude': 10, 'longitude': 350}

test_ds_rolled = dew_point_xr.roll(roll_test, roll_coords=True)

test_ds_rolled

test_ds = test_ds_rolled['d2m'].sel(latitude=slice(80, 40), longitude=slice(350.25, 10.25))
test_ds

test_ds = test_ds_rolled['d2m'].isel(time=0)
test_ds

<xarray.DataArray 'd2m' (latitude: 721, longitude: 1440)>
array([[242.3083 , 242.3083 , 242.3083 , ..., 242.3083 , 242.3083 ,
        242.3083 ],
       [242.78452, 242.78452, 242.78629, ..., 242.781  , 242.781  ,
        242.78276],
       [243.4769 , 243.47865, 243.48216, ..., 243.46635, 243.47162,
        243.47337],
       ...,
       [245.03384, 245.0356 , 245.03912, ..., 245.02155, 245.02505,
        245.02858],
       [245.15686, 245.16037, 245.16212, ..., 245.14807, 245.15158,
        245.1551 ],
       [245.23593, 245.23593, 245.23593, ..., 245.23593, 245.23593,
        245.23593]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 272.5 272.8 273.0 273.2 ... 271.8 272.0 272.2
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    time       datetime64[ns] 1990-01-01
Attributes:
    units:      K
    long_name:  2 metre dewpoint temperature

In [27]:
#print(closest_lon_w, closest_lon_e, closest_lat_s, closest_lat_n)
temp_ds

array([[274.203  , 274.022  , 273.94995, ..., 275.81442, 275.91635,
        275.96906],
       [275.60004, 275.42606, 275.21518, ..., 275.832  , 275.98135,
        275.97433],
       [275.61234, 275.55084, 275.44186, ..., 275.96906, 276.04288,
        276.01474],
       ...,
       [274.84616, 274.811  , 274.7794 , ..., 280.80157, 280.94745,
        280.94394],
       [274.58783, 274.67218, 274.51404, ..., 280.94043, 280.99313,
        280.91406],
       [274.35062, 274.3805 , 274.45956, ..., 280.98083, 280.94568,
        280.8596 ]], dtype=float32)

In [35]:
# check if each square has the same shape

for i in range(0, len(var_out_6)):
    print(var_out_6[i].shape)

(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)


In [36]:
var_out_6[15]

array([[255.66893, 254.88518, 254.39137, ..., 266.1845 , 265.65558,
        264.90344],
       [256.05377, 255.34383, 255.05212, ..., 265.93497, 265.5097 ,
        265.06863],
       [256.5511 , 255.93779, 255.65135, ..., 265.81372, 265.36914,
        264.91925],
       ...,
       [268.6992 , 268.9575 , 269.24042, ..., 270.15247, 269.72543,
        269.38803],
       [269.30545, 269.52686, 269.7553 , ..., 269.88535, 269.4759 ,
        269.34763],
       [269.62527, 269.9732 , 270.25613, ..., 269.42142, 269.17892,
        269.0963 ]], dtype=float32)

In [13]:
# mean, min, max value of each time step

mean_out_d2m = []
min_out_d2m = []
max_out_d2m = []

for i in range(0, len(var_out)):
    mean_out_d2m.append(np.mean(var_out[i]))
    min_out_d2m.append(np.min(var_out[i]))
    max_out_d2m.append(np.max(var_out[i]))

In [14]:
# save as csv

df_mean = pd.DataFrame(mean_out_d2m)
df_min = pd.DataFrame(min_out_d2m)
df_max = pd.DataFrame(max_out_d2m)

df_mean.to_csv('d2m_mean.csv')
df_min.to_csv('d2m_min.csv')
df_max.to_csv('d2m_max.csv')

For all variables, (class by variable (folder), then by storm)) 